In [24]:
import openai
import re
import streamlit as st
import os
import sqlalchemy
import json
import requests
import pandas as pd

from langchain.memory import StreamlitChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory

from langchain.agents import create_sql_agent, AgentExecutor
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.prompts import PromptTemplate

from langchain.callbacks import StreamlitCallbackHandler, HumanApprovalCallbackHandler, FileCallbackHandler, ContextCallbackHandler, LLMonitorCallbackHandler
#st.set_page_config(page_title="Kai SQL Bot", page_icon=":robot_face:")

from src.workspace_connection.workspace_connection import connect_to_snowflake
from few_shot_examples import custom_tool_list
from prompts import frosty_gen_sql, custom_gen_sql

openai.api_key = st.secrets.OPENAI_API_KEY


llm_monitor_callback = LLMonitorCallbackHandler(app_id=st.secrets.LUNARY_APP_ID)


st.session_state["chat_messages"] = []

#msgs = StreamlitChatMessageHistory(key="chat_messages")
memory = ConversationBufferMemory()

llm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0,streaming=True)

    
def initialize_connection():
    account_identifier = st.secrets["account_identifier"]
    user = st.secrets["user"]
    password = st.secrets["password"]
    database_name = st.secrets["database_name"]
    schema_name = st.secrets["schema_name"]
    warehouse_name = st.secrets["warehouse_name"]
    role_name = st.secrets["user"]
    conn_string = f"snowflake://{user}:{password}@{account_identifier}/{database_name}/{schema_name}?warehouse={warehouse_name}&role={role_name}"
    db = SQLDatabase.from_uri(conn_string)
    toolkit = SQLDatabaseToolkit(llm=llm, db=db, view_intermediate_results=True, view_messages=True)
    agent_executor = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True,
        handle_parsing_errors=True,
        max_iterations=50,
        extra_tools=custom_tool_list,
        agent_type=AgentType.OPENAI_FUNCTIONS,
        memory=memory,
        return_intermediate_steps=True, 
        #human_callback=HumanApprovalCallbackHandler(),
        #suffix=custom_suffix
    )
    return agent_executor, conn_string
  

agent_executor, conn_string = initialize_connection()   


In [25]:
prompt = "Help me write a query to calculate the number of new customers by month, as well as the the the magnitude of the change for new customers (50, -28, etc.) from month to month and the percentage of the change of new customers (+4%, -8%, etc) from month to month. "

from langchain.callbacks import FileCallbackHandler, ContextCallbackHandler

from loguru import logger

# automatically generate a log file name based on the current datetime
from datetime import datetime
now = datetime.now()
current_time = now.utcnow().strftime("%Y%m%d%H%M%S")
logfile = f"logs/{current_time}.log"

logger.add(logfile, colorize=True, enqueue=True)
handler = FileCallbackHandler(logfile)


prompt_formatted = custom_gen_sql.format(context=prompt)
try:
    response = agent_executor.run(input=prompt_formatted, callbacks=[llm_monitor_callback, handler], memory=memory, return_intermediate_steps=True, verbose=True)
except ValueError as e:
    response = str(e)
    if not response.startswith("Could not parse LLM output: `"):
        raise e
    response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


customer, customer_addresses, inventory_items, inventory_levels, line_item, locations, order, product, product_images, product_options, product_variant
Invoking: `sql_db_schema` with `customer, order`



CREATE TABLE "customer" (
	"tax_exempt" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__country" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"marketing_opt_in_level" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__customer_id" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"updated_at" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__company" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"sms_marketing_consent__consent_updated_at" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"orders_count" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__province_code" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__default" VARCHAR(16777216) NOT NULL DEFAULT '', 
	"default_address__zip"

AttributeError: 'str' object has no attribute 'intermediate_steps'

In [1]:
!pip install context-python --upgrade

  Obtaining dependency information for context-python from https://files.pythonhosted.org/packages/86/f4/8bbf99e1e6f59e4afb80e365952eb4f9de4b75a8a652306a6259f448f98b/context_python-0.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for azure-core<2.0.0 from https://files.pythonhosted.org/packages/b0/e2/b6cdd23d8d9cc430410cc309879883aff67736c02528cd1fdc07c48158b1/azure_core-1.29.6-py3-none-any.whl.metadata
  Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 9.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
